In [ ]:
import requests
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input

# Define the URL for the API
url = "https://divulgacandcontas.tse.jus.br/divulga/rest/v1/candidatura/listar/2022/SP/2030402021/2/candidatos"

# Send a GET request to the API to retrieve the data
response = requests.get(url)

# Load the JSON data into a DataFrame
df = pd.DataFrame(response.json())

# Filter the DataFrame to keep only the relevant columns
df = df[['sqCandidato', 'sgPartido', 'vrDeclarado']]

# Rename the columns for clarity
df = df.rename(columns={'sqCandidato': 'SQ_CANDIDATO', 'sgPartido': 'SG_PARTIDO', 'vrDeclarado': 'VR_DESPESA_MAX_CAMPANHA'})

# Group by political party and sum the campaign budget
party_budget = df.groupby('SG_PARTIDO')['VR_DESPESA_MAX_CAMPANHA'].sum().reset_index()

# Sort by budget in descending order
party_budget = party_budget.sort_values('VR_DESPESA_MAX_CAMPANHA', ascending=False)

# Calculate the proportion of each party's budget
total_budget = party_budget['VR_DESPESA_MAX_CAMPANHA'].sum()
party_budget['Proportion'] = party_budget['VR_DESPESA_MAX_CAMPANHA'] / total_budget

# Define the Plotly Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div(children=[
    html.H1(children=f"2022 Brazilian Federal Elections - Political Parties' Campaign Budgets ({total_budget:,.2f} BRL)", style={'textAlign': 'center'}),
    dcc.Graph(
        id='waffle-chart',
        figure=go.Figure(
            go.Waffle(
                values=party_budget['Proportion'] * 100, 
                labels=party_budget['SG_PARTIDO'], 
                rows=10, 
                colors=['#006600', '#E30B5D', '#FF8C00', '#0057AE', '#FDB813', '#DC143C', '#708090', '#191970', '#228B22', '#D3D3D3', '#1E90FF', '#9ACD32', '#FF69B4', '#CD5C5C', '#FFD700', '#9400D3', '#8B008B', '#A52A2A', '#FF4500', '#808080', '#FF1493', '#7CFC00', '#FF00FF', '#00FFFF', '#FFFAFA', '#FFFF00', '#00FF00', '#0000FF', '#FFC0CB', '#800000'],
                title={
                    'text': "Political Parties' Campaign Budgets",
                    'font': {'size': 18}
                },
                legend={
                    'title': {'text': 'Party', 'font': {'size': 14}},
                    'traceorder': 'normal',
                    'font': {'size': 12},
                    'x': -0.1,
                    'y': 1.2
                },
                icon_size=24,
                icon='square'
            ),
            layout={
                'margin': {'l': 0, 'r': 0, 't': 
